In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [3]:

df = pd.read_csv('axisbank.csv')

df.set_index('datetime', inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
df_scaled = scaler.fit_transform(df)
lookback_window = 5
features = df.shape[1]
def create_sequences(data, lookback_window):
  X, y = [], []
  for i in range(len(data) - lookback_window - 1):
    X.append(data[i:(i + lookback_window), :])
    y.append(data[i + lookback_window, :])
  return np.array(X), np.array(y)

X, y = create_sequences(df_scaled, lookback_window)
X_train, y_train = X, y  # Replace with train/validation split if desired
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(units=features))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=50,batch_size=30)
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]
for _ in range(number_of_predictions):
  next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
  next_data_point_original = scaler.inverse_transform(next_data_point)
  forecasted_values.append(next_data_point_original.flatten())
  last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)
forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(),forecast_df['close'].min())


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.1991
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0382
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0247
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0137
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0087
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0073
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0077
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0077
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0064
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0065
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0077
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0066
Epoch 13/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0067
Epoch 14/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0070
Epoch 15/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0070
Epoch 16/50
8/8 ━━━━━━━━━━━━━━━━━━